# Segmentación con kmedias

Usaremos datos tomados del proyecto de kaggle *Credit Card Dataset for Clustering*:

In [ ]:
%autosave 0
import pandas as pd
import matplotlib.pyplot as plt
from plotnine import *
import numpy as np
# leer datos 
general = pd.read_csv("../datos/CC-GENERAL.csv")
general.info()

Datos tomados de [kaggle](https://www.kaggle.com/arjunbhasin2013/ccdata). Las variables ya tienen algo de procesamiento previo. Podemos crear también nuevas variables que consideremos puedan ser informativas para hacer los segmentos.

- CUST_ID : Identification of Credit Card holder (Categorical) 
- BALANCE : Balance amount left in their account to make purchases  
- BALANCE_FREQUENCY : How frequently the Balance is updated, score between 0 and 1 (1 = frequently updated, 0 = not frequently updated) 
- PURCHASES : Amount of purchases made from account 
- ONEOFF_PURCHASES : Maximum purchase amount done in one-go
- INSTALLMENTS_PURCHASES : Amount of purchase done in installment 
- CASH_ADVANCE : Cash in advance given by the user 
- PURCHASES_FREQUENCY : How frequently the Purchases are being made, score between 0 and 1 (1 = frequently purchased, 0 = not frequently purchased) 
- ONEOFFPURCHASESFREQUENCY : How frequently Purchases are happening in one-go (1 = frequently purchased, 0 = not frequently purchased) 
- PURCHASESINSTALLMENTSFREQUENCY : How frequently purchases in installments are being done (1 = frequently done, 0 = not frequently done)
- CASHADVANCEFREQUENCY : How frequently the cash in advance being paid 
- CASHADVANCETRX : Number of Transactions made with "Cash in Advanced" 
- PURCHASES_TRX : Numbe of purchase transactions made 
- CREDIT_LIMIT : Limit of Credit Card for user 
- PAYMENTS : Amount of Payment done by user 
- MINIMUM_PAYMENTS : Minimum amount of payments made by user 
- PRCFULLPAYMENT : Percent of full payment paid by user 
- TENURE : Tenure of credit card service for user

In [ ]:
general.hist(figsize=(20, 20), bins = 50)
plt.show()


## 1. Preprocesamiento

In [ ]:
# ponemos en minúsculas las variables y calculamos resúmenes:
general.columns = [x.lower() for x in general.columns.tolist()]
general.describe()

In [ ]:
#faltantes
general.isnull().sum()

In [ ]:
print(pd.crosstab(general["minimum_payments"].isnull(), general["payments"] > 0))
# rellenar con 0 en minimum payments
general['minimum_payments'].fillna(value = 0, inplace = True)
# eliminar caso sin limite de cŕedito
general.dropna(axis = 0, inplace = True)

In [ ]:
general.describe()

Los datos presentan asimetría y colas largas. Puede ser una buena idea transformar a logaritmo las variables positivas. Esto implica que nos interesan diferencias entre los casos **multiplicativas** en lugar de **aditivas**, que es más apropiado aquí.

In [ ]:
vars_pos = ["balance", "purchases", "oneoff_purchases", "installments_purchases", "cash_advance", "cash_advance_trx",
          "purchases_trx", "credit_limit", "payments", "minimum_payments"]
general_trans = general.copy()
for var in vars_pos:
    general_trans[var + "_log"] = np.log10(1 + general_trans[var])
general_trans

In [ ]:
# seleccionar variables para segmentar:
vars_segmentos = ["purchases_log", "oneoff_purchases_log", "installments_purchases_log", "cash_advance_log", 
                  "credit_limit_log", "payments_log", "minimum_payments_log", "balance_frequency", "purchases_frequency", 
                  "oneoff_purchases_frequency", "purchases_installments_frequency", "prc_full_payment"]
general_s = general_trans[vars_segmentos]
general_s

Estandarizamos, pues las variables están en distintas escalas

In [ ]:
from sklearn import preprocessing
std_scaler = preprocessing.StandardScaler()
x_escalada = std_scaler.fit_transform(general_s)
general_esc = pd.DataFrame(x_escalada)
general_esc.columns = general_s.columns
general_esc.round(2)

## 3. Segmentación por k-medias

Usamos k-medias para construir varias soluciones

In [ ]:
from sklearn.cluster import KMeans
# ajustar semilla para que sea reproducible
np.random.seed(211)
inercia = []
num_clusters = range(1, 15)
for i in num_clusters:
    agrupador = KMeans(# sustituye tu código aquí)
    kmedias = agrupador.fit(#sustituye los datos)
    inercia.append(kmedias.inertia_)

# plot
inercia_df = pd.DataFrame({"inercia":inercia, "num_clusters":num_clusters})


In [ ]:
inercia_df
# haz gráfica de codo

**Pregunta**: Hay varias soluciones que podemos probar. ¿Qué indica la gráfica de codo?

## 4. Agrupar y perfilar

Calculamos la segmentación y vemos cuántos clientes caen en cada grupo:

In [ ]:
agrupador = KMeans(# rellena tu código)
agrupador_ajustado = agrupador.fit(general_esc)
grupos = agrupador_ajustado.predict(general_esc)
# calcula tamaño de cada grupo en crudo y porcentajes

Checa convergencia de solución (si obtienes un valor igual a max_iter, puedes iterar más veces):

In [ ]:
agrupador_ajustado.n_iter_

Ahora vamos a perfilar en las variables que usamos para segmentar, que están estandarizadas:

In [ ]:
def perfilar(general_esc, grupos, tipo = "aditivo"):
    ### producir perfiles aditivos o multiplicativos según grupos
    # convertimos a categoría
    datos = general_esc.copy()
    datos["grupo"] = pd.Series(grupos).astype("category")
    # calculamos medias por grupo de las variables
    agregados = datos.groupby("grupo").mean()
    # pivoteamos las variables a forma larga
    agregados_larga = agregados.reset_index() \
        .melt(id_vars = ["grupo"])
    # ahora calculamos medias totales a lo largo de grupos
    medias = agregados_larga.drop(columns=["grupo"]).groupby("variable").mean() \
        .rename(columns = {"value":"media"})
    # en estas líneas tomamos las medias por grupo y les agregamos
    # la medias a total:
    variable_cat = pd.Categorical(agregados_larga['variable'], 
        categories=agregados.columns.tolist())
    agregados_larga = agregados_larga.assign(variable_cat = variable_cat). \
        merge(medias, on = "variable", how = "left")
    # calculamos el perfil (diferencia vs media)
    if tipo=="aditivo":
        agregados_larga["perfil"] = agregados_larga["value"] - agregados_larga["media"]
    else:
        agregados_larga["perfil"] = 100 * (agregados_larga["value"] / agregados_larga["media"] - 1.0)
    return agregados_larga

agregados_larga = perfilar(general_esc, grupos)
agregados_larga

In [ ]:
# pivoteamos los grupos para obtener una tabla más legible
agregados_larga[['grupo', 'variable_cat', 'value']]. \
    pivot(columns = 'grupo', values='value', index = "variable_cat").round(2)

In [ ]:
# y usamos formato condicional para leer más facilmente
def color_negative_red(val):
    color = 'red' if val < -0.3 else 'black' if val > 0.3 else 'gray'
    return 'color: %s' % color

def tabla_perfiles(agregados_larga, columna, renglon):
    resumen_perfil = agregados_larga[['grupo', 'variable_cat', 'perfil']]. \
        pivot(columns='grupo', values='perfil', index="variable_cat").round(2). \
        sort_values(by=renglon, axis=1). \
        sort_values(by=columna, axis=0)
    return resumen_perfil.style.applymap(color_negative_red)

tabla_perfiles(agregados_larga, columna = 2, renglon = 'purchases_log')

Estos son perfiles aditivos pero en escala logarítmica estandarizada. Nos dan una idea, pero pueden ser difíciles de interpetar.

### Prueba de permutaciones (optativo pero útil)

Para entender qué tan grandes son estas diferencias, podemos hacer una prueba de permutaciones. Permutamos los grupos y comparamos la variación contra la observada:

In [ ]:
agregados_larga_perm = perfilar(general_esc, grupos[np.random.permutation(grupos.size)])
tabla_perfiles(agregados_larga_perm, columna = 2, renglon = 'purchases_log')

Esta tabla indica que típicamente podemos considerar diferencias de alrededor de +/-0.10 son producidas por la agrupación que construimos.

## 4. Interpretación y nombres de grupos

Podemos perfilar **en las variables originales** que usamos para interpretar y nombrar grupos, pero usamos **diferencias multiplicativas**:

In [ ]:
general_vars = list(map(lambda x: x.replace('_log', ''), vars_segmentos))
print(general_vars)
agregados_larga_m = perfilar(general.loc[:, general_vars], grupos, tipo = "mult")
with pd.option_context('display.precision', 3):
    tab = tabla_perfiles(agregados_larga_m.round(0), columna = 2, renglon = 'purchases')
tab

Ahora podemos empezar a nombrar grupos. ¿Qué mejores nombres propondrías?

In [ ]:
nombres = # rellena tu código con nombres apropiados {2:'a', 3:'b', 4:'c', 0:'d', 1:'f'}
grupos_nombre = pd.Series(grupos).replace(nombres)
agregados_larga_m['grupo'].replace(nombres, inplace = True)
with pd.option_context('display.precision', 3):
    tab = tabla_perfiles(agregados_larga_m.round(0), 
                         columna = 'a', renglon = 'purchases')
tab

**Pregunta**: considerarías que algunos grupos son muy similares y valdría la pena usar una solución de menos grupos. ¿Cómo se ve una solución de 4 o 6 grupos?

**Ejercicio**: repite con otro número de grupos, y considera ventajas y desventajas en cuanto a la interpretación.



## Perfilamiento en variables suplementarias

Ahora perfilamos usando otras variables que no usamos en la segmentación. 

In [ ]:
suplementarias = ['balance', 'tenure']
agregados_larga_m = perfilar(general.loc[:, suplementarias], grupos_nombre, tipo = "mult")
with pd.option_context('display.precision', 5):
    tab = tabla_perfiles(agregados_larga_m.round(2), columna = "a", renglon = 'tenure')
tab

In [ ]:
usuarios_n = grupos_nombre.value_counts()
usuarios_n

In [ ]:
usuarios_pct = 100*usuarios_n/sum(usuarios_n)
usuarios_pct.round(1)

Este resultado no es muy útil